# Compute the spectrum of Fe X

In [2]:
using JAC

Multiply charged ions have attracted contineous interest due to recent observations by the Solar and heliospheric Observatory (SOHO), the Far Ultraviolet Spectroscopic Explorer (FUSE) and several other astro physical missions. In the extreme ultra-violet (EUV) range of spectra, for instance, many of the spectroscopically observed lines belong to the iron-group elements owing to their high abundance in different objects. Until the present, however, several of these lines remained unclassified because reliable theoretical predictions are often not available that consistently include relativistic and correlation effects. 

To demonstrate the use of the JAC toolbox, we here consider the level structure of Fe$^{9+}$ ions which, in spectroscopic notation, give rise to the spectrum Fe X. Several medium-to-large scale computations have been carried out to calculate the excitation energies and transition probabilities of (and among) the levels from the
[Ne] $3s^2 3p^5$ -> $3s^2 3p^4 3d + 3s 3p^6$ configuration. These three configurations comprises in total
31 fine-structure levels with total angular momenta $J = 1/2, ..., 9/2$ (cf. Dong et al., 1999). Although the 
$3s^2 3p^4 3d + 3s 3p^6$ levels mainly decay by electric-dipole (E1) transitions, also M1, E2 and M2 multipole transitions have been considered in the literature. Below, we show how this *complete* Fe X spectrum can be calculated approximately by just a single call within the JAC toolbox.



However, before we shall perform this computation, let us define a standard grid for the computations:

In [3]:
grid = JAC.Radial.Grid("grid: exponential")
JAC.define("standard grid", grid)

Define a radial grid of type MeshGL with 400 grid points
 [rnt=2.000e-06, h=5.000e-02, hp=0.000e+00, NoPoints=390, r_max=9.161e+02;
  B-splines wit break points at every 7th point, nsL=56, nsS=57, orderL=7, orderS=8, orderGL=7] 
Re-define the standard grid with 390 grid points.


Moreover, we can show here of how to select proper **physical units**, in which all the energies, rates, etc. are handled at input and output time. To see the present `settings` of this units, we can simply call:

In [4]:
JAC.display("settings")

Current settings of the JAC module:  
-----------------------------------  

  + Framework:                              relativistic
  + Energy unit:                            eV
  + Rate and transition probability unit:   1/s
  + Cross section unit:                     barn
  + Time unit:                              sec

  + A standard grid has been defined; cf. JAC.give()



Of course, we could easily change any of these units by a proper call to `JAC.define()`, but we shall leave the present (standard) choice for the given computations. Call `JAC.define()` to change these settings:


In [5]:
? JAC.define

`JAC.define()`  ... (re-) defines some 'standard' settings which are common to all the computations with the JAC module, and which can                      be 'overwritten' by the user. –- An improper setting of some variable may lead to an error message, if recognized                     immediately. The following defaults apply if not specified otherwise by the user: the framework is 'relativistic',                      energies are given in eV and cross sections in barn. Note that, internally, atomic units are used throughout for                      all the computations within the program. nothing is returned if not indicated otherwise.

  * `("framework: relativistic")`  or  `("framework: non-relativistic")`   ... to define a relativistic or non-relativistic framework                 for all subsequent computations.
  * `("method: continuum, spherical Bessel")`  or  `("method: continuum, pure sine")`  or  `("method: continuum, asymptotic Coulomb")`                  or  `("method: continuum, nonrelativistic Coulomb")`  or  `("method: continuum, Galerkin")`                  ... to define a a method for the generation of the continuum orbitals as (pure) spherical Bessel, pure sine,                asymptotic Coulomb, nonrelativistic Coulomb orbital or by means of the B-spline-Galerkin method.
  * `("method: normalization, pure sine")`  or  `("method: normalization, pure Coulomb")`   ... to define a method for the                 normalization of the continuum orbitals as asymptotically (pure) sine or Coulomb functions.
  * `("QED model: Petersburg")`  or  `("QED model: Sydney")`   ... to define a model for the computation of the QED corrections following                the work by Shabaev et al. (2011; Petersburg) or Flambaum and Ginges (2004; Syney).
  * `("unit: energy", "eV")`  or  `("unit: energy", "Kayser")`  or  `("unit: energy", "Hartree")`  or  `("unit: energy", "Hz")`  or   `("unit: energy", "Hz")`  ... to (pre-) define the energy units for all further printouts and communications with the JAC module.
  * `("unit: cross section", "a.u.")`  or  `("unit: cross section", "barn")`  or  `("unit: cross section", "Mbarn")`  ... to (pre-)                 define the unit for the printout of cross sections.
  * `("unit: rate", "a.u.")`  or  `("unit: rate", "1/s")`  ... to (pre-) define the unit for the printout of rates.
  * `("unit: time", "a.u.")`  or  `("unit: time", "sec")`  or  `("unit: time", "fs")`  or  `("unit: time", "as")`  ... to (pre-) define                 the unit for the printout and communications of times with the JAC module.

---

  * `("relativistic subshell list", subshells::Array{Subshell,1}; printout::Bool=true)`                   ... to (pre-) define internally the standard relativistic subshell list on which the standard order of orbitals is based.

---

  * `("standard grid", grid::Radial.Grid)`  ... to (pre-) define internally the standard radial grid which is used to represent most orbitals.

---

  * `("QED: damped-hydrogenic", Znuc::Float64, wa::Array{Float64,1})`  ... to (re-) define the lambda-C damped overlap integrals of                              the lowest kappa-orbitals [ wa*1s*1/2, wa*2p*1/2, wa*2p*3/2, wa*3d*3/2, wa*3d*5/2 ] for the (new)                              nuclear charge Znuc; nothing is returned.



Apparently, we can call `JAC.define()` also in order to open some ASCII (.txt) file into which all the final (important) results are printed for later reference and inspection.


In [6]:
JAC.define("print summary: open", "06-output-radiative.txt")


Next, we shall now describe the computation to be carried out. Apart from a title line (String) and nuclear model (cf. tutorial 03-setting-the-nucleus.ipynb), we here need to specify the initial- and final-state configurations that are to be included into the computation. This is done by defining an instance of an:

```julia
comp = Atomic.Computation("Energies and Einstein coefficients for the spectrum Fe X",  Nuclear.Model(26.);              
              initialConfigs = [Configuration("[Ne] 3s 3p^6"), Configuration("[Ne] 3s^2 3p^4 3d")],              
              finalConfigs   = [Configuration("[Ne] 3s^2 3p^5")],               
              process = RadiativeX, 
              processSettings = Radiative.Settings(...) ); 
```

In principle, any number of configurations can be specified as long as they are listed explicitly. Here, we have selected also the identifier `RadiativeX` to consider the computation of transition probabilities between the levels of the three configurations given above. All further details about any computation can be specified by means of some proper `settings` that are specific to each particular process, here `Radiative.Settings()`. To better understand to which extent we can affect and control these computations, let us look at the definition of this data struct:


In [7]:
? Radiative.Settings

`struct  Radiative.Settings`  ... defines a type for the details and parameters of computing radiative lines.

```
+ multipoles              ::Array{EmMultipoles}     ... Specifies the (radiat. field) multipoles to be included.
+ gauges                  ::Array{UseGauge}         ... Gauges to be included into the computations.
+ calcAnisotropy          ::Bool                    ... True, if the anisotropy (structure) functions are to be 
                                                        calculated and false otherwise 
+ printBeforeComputation  ::Bool                    ... True, if all energies and lines are printed before comput.
+ selectLines             ::Bool                    ... True, if lines are selected individually for the computat.
+ selectedLines           ::Array{Tuple{Int64,Int64},1}  ... List of lines as tupels (inital-level, final-level).
+ photonEnergyShift       ::Float64                 ... An overall energy shift for all photon energies.
+ mimimumPhotonEnergy     ::Float64                 ... minimum transition energy for which (photon) transitions 
                                                        are included into the computation.
+ maximumPhotonEnergy     ::Float64                 ... maximum transition energy for which (photon) transitions 
                                                        are included.
```

---

`JAC.Radiative.Settings()`  ... constructor for the default values of radiative line computations


Of course, we can always apply the default values for any particular compuation which, for present case, are given by:

In [8]:
JAC.Radiative.Settings()

Radiative.Settings(with multipoles = EmMultipole[E1], use-gauges = UseGauge[Coulomb], calcAnisotropy = false, 
                 printBeforeComputation = false, selectLines = false, selectedLines = Tuple{Int64,Int64}[], photonEnergyShift = 0.0 
                 mimimumPhotonEnergy = 0.0, maximumPhotonEnergy = 0.0) 



Below, we wish to consider also other multipole transitions, namely E1, M1, E2, M2, and like perform all calculations within both, the Coulomb and Babushkin gauges. In addition, we shall **NOT** compute the (angular) anisotropy parameters nor do we wish to select individual transitions right from the beginning. This latter option might be interesting if large sets of configurations are to be considered. Moreover, we do not neither specify any (overall) shift nor restrict the photon energies, two features that are of particular interest for large data sets


We these remarks in mind, we now specify the atomic computation in the form:


In [9]:
comp = Atomic.Computation("Energies and Einstein coefficients for the spectrum Fe X",  Nuclear.Model(26.);
              initialConfigs = [Configuration("[Ne] 3s 3p^6"), Configuration("[Ne] 3s^2 3p^4 3d")],
              finalConfigs   = [Configuration("[Ne] 3s^2 3p^5")], 
              process = RadiativeX, 
              processSettings = Radiative.Settings([E1, M1, E2, M2], [JAC.UseCoulomb, JAC.UseBabushkin], 
                        false, true, false, Tuple{Int64,Int64}[], 0., 0., 10000. ) ); 

shellOccList = SubString{String}["[Ne]", "3s", "3p^6"]
shellOccList = SubString{String}["[Ne]", "3s^2", "3p^4", "3d"]
shellOccList = SubString{String}["[Ne]", "3s^2", "3p^5"]
Define a radial grid of type MeshGL with 400 grid points
 [rnt=2.000e-06, h=5.000e-02, hp=0.000e+00, NoPoints=390, r_max=9.161e+02;
  B-splines wit break points at every 7th point, nsL=56, nsS=57, orderL=7, orderS=8, orderGL=7] 



All what is needed now is to **perform** this computation by the call:


In [10]:
perform(comp)


... in perform('computation: SCF', ...
perform-aa: Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^1 3p_1/2^2 3p_3/2^4 
perform-aa: Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_3/2^4 3d_5/2^1 
perform-aa: Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^1 3p_3/2^3 3d_5/2^1 
perform-aa: Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^2 3p_3/2^2 3d_5/2^1 
perform-aa: Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_3/2^4 3d_3/2^1 
perform-aa: Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^1 3p_3/2^3 3d_3/2^1 
perform-aa: Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^2 3p_3/2^2 3d_3/2^1 
(Re-) Define a new standard subshell list.
(Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 26.0 with mass = 55.38, radius R = 3.756573410087755 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Gener

  1s_1/2::  en [a.u.] = -2.50257975e+02;   self-consistency = 1.536101e-01  [1.000000e+02 for symmetry block kappa = -1]
  2s_1/2::  en [a.u.] = -3.13236973e+01;   self-consistency = 4.636006e-01  [1.000000e+02 for symmetry block kappa = -1]
  3s_1/2::  en [a.u.] = -8.60660281e+00;   self-consistency = 6.298722e-01  [1.000000e+02 for symmetry block kappa = -1]
  2p_1/2::  en [a.u.] = -2.85749503e+01;   self-consistency = 4.988781e-01  [1.000000e+02 for symmetry block kappa = 1]
  3p_1/2::  en [a.u.] = -7.91990546e+00;   self-consistency = 6.542990e-01  [1.000000e+02 for symmetry block kappa = 1]
  2p_3/2::  en [a.u.] = -3.02004215e+01;   self-consistency = 4.742788e-01  [1.000000e+02 for symmetry block kappa = -2]
  3p_3/2::  en [a.u.] = -8.31380968e+00;   self-consistency = 6.383937e-01  [1.000000e+02 for symmetry block kappa = -2]
  3d_3/2::  en [a.u.] = -7.17024977e+00;   self-consistency = 6.801781e-01  [1.000000e+02 for symmetry block kappa = 2]
  3d_5/2::  en [a.u.] = -7.25812748

Compute CI matrix of dimension 8 x 8 for the symmetry 5/2^+ ...   ... done.
Compute CI matrix of dimension 2 x 2 for the symmetry 9/2^+ ...   ... done.
Compute CI matrix of dimension 5 x 5 for the symmetry 7/2^+ ...   ... done.
Compute CI matrix of dimension 8 x 8 for the symmetry 3/2^+ ...   ... done.
Compute CI matrix of dimension 6 x 6 for the symmetry 1/2^+ ...   ... done.

  Eigenenergies:

  Level  J Parity          Hartrees                    eV                   [eV]

     1    1/2 +     -1.241474818158662e+03    -3.378225051102466e+04    -3.378225051102466e+04 
     2    7/2 +     -1.241013110079942e+03    -3.376968679426605e+04    -3.376968679426605e+04 
     3    5/2 +     -1.241013074627370e+03    -3.376968582955244e+04    -3.376968582955244e+04 
     4    3/2 +     -1.241007439938115e+03    -3.376953250184801e+04    -3.376953250184801e+04 
     5    1/2 +     -1.241000331020361e+03    -3.376933905834283e+04    -3.376933905834283e+04 
     6    9/2 +     -1.240876002318339e

... in perform('computation: SCF', ...
perform-aa: Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^1 3p_3/2^4 
perform-aa: Configuration: 1s_1/2^2 2s_1/2^2 2p_1/2^2 2p_3/2^4 3s_1/2^2 3p_1/2^2 3p_3/2^3 
(Re-) Define a new standard subshell list.
(Re-) Define a storage array for various B-spline matrices:
Nuclear model = Fermi nuclear model for Z = 26.0 with mass = 55.38, radius R = 3.756573410087755 fm and nuclear spin I = 0, dipole moment mu = 0.0 and quadrupole moment Q = 0.0. 
Generate hydrogenic orbital for subshell 1s_1/2 
  -----------------------------------------------------------------------------
   Index    Subshell     Energies [a.u.]    Dirac-E  [a.u.]     Delta-E / |E|    
  -----------------------------------------------------------------------------
      1      1s_1/2      -3.41095882e+02    -3.41097837e+02    +5.73282577e-06    
      2      2s_1/2      -8.54687335e+01    -8.54689584e+01    +2.63136714e-06    
      3      3s_1/2      -3.78995149e+01

  1s_1/2::  en [a.u.] = -2.64591459e+02;   self-consistency = 7.576627e-04  [4.923840e-02 for symmetry block kappa = -1]
  2s_1/2::  en [a.u.] = -3.81286960e+01;   self-consistency = 2.654989e-03  [4.923840e-02 for symmetry block kappa = -1]
  3s_1/2::  en [a.u.] = -1.03884434e+01;   self-consistency = 3.132303e-03  [4.923840e-02 for symmetry block kappa = -1]
  2p_1/2::  en [a.u.] = -3.40261735e+01;   self-consistency = 1.576825e-03  [7.286212e-03 for symmetry block kappa = 1]
  3p_1/2::  en [a.u.] = -9.13544931e+00;   self-consistency = 1.666520e-03  [7.286212e-03 for symmetry block kappa = 1]
  2p_3/2::  en [a.u.] = -3.35588298e+01;   self-consistency = 3.685408e-04  [8.962864e-03 for symmetry block kappa = -2]
  3p_3/2::  en [a.u.] = -9.05569379e+00;   self-consistency = 3.503784e-04  [8.962864e-03 for symmetry block kappa = -2]

Iteration 5 for symmetries ... 
  1s_1/2::  en [a.u.] = -2.64543260e+02;   self-consistency = 9.109065e-05  [4.985591e-03 for symmetry block kappa = -1]
 

Compute radiative E1 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [1-1] ... nz = 2
done. 
Compute radiative E1 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [1-1] ... done. 
Compute radiative M2 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [1-1] ... done. 
Compute radiative E1 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [1-2] ... nz = 2
done. 
Compute radiative E1 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [1-2] ... done. 
Compute radiative M2 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [2-1] ... done. 
Compute radiative E1 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [3-1] ... done. 
Compute radiative E1 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [3-1] ... done. 
Compute ra

Compute radiative M2 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [20-1] ... done. 
Compute radiative M2 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [20-2] ... done. 
Compute radiative E1 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [21-1] ... done. 
Compute radiative E1 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [21-1] ... done. 
Compute radiative M2 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [21-1] ... done. 
Compute radiative M2 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [21-2] ... done. 
Compute radiative M2 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [22-1] ... done. 
Compute radiative E1 matrix of dimension 2 x 29 in the initial- and final-state bases for the transition [23-1] ... done. 
Compute radiativ

       15 --     1      7/2 + --> 3/2 -     5.42573198e+01        M2       Magnetic     1.12619149e+01  1.74557325e-09    1.76325860e-10    7.41272702e-15      
       16 --     1      5/2 + --> 3/2 -     5.42958658e+01        E1       Coulomb      6.03687420e+07  7.00283581e-03    7.07881076e-04    3.97354275e-08      
       16 --     1      5/2 + --> 3/2 -     5.42958658e+01        E1       Babushkin    7.55452949e+07  8.76333147e-03    8.85840634e-04    4.97248160e-08      
       16 --     1      5/2 + --> 3/2 -     5.42958658e+01        M2       Magnetic     4.76920476e+01  5.53232630e-09    5.59234745e-10    3.13914758e-14      
       16 --     2      5/2 + --> 1/2 -     5.22806325e+01        M2       Magnetic     6.58789872e-01  1.71205029e-10    1.66639113e-11    4.33623368e-16      
       17 --     1      5/2 + --> 3/2 -     5.46600496e+01        E1       Coulomb      1.32053434e+08  1.50141842e-02    1.52788743e-03    8.69191482e-08      
       17 --     1      5/2 + --> 

 
  Radiative lifetimes (as derived from these computations):
 
  ---------------------------------------------------------------------------------------------------------------
     Level        J^P        Level energy      Used Gauge                  Lifetime                Decay widths     
                                 [eV]                               [a.u.]          [sec]             [eV]         
  ---------------------------------------------------------------------------------------------------------------
         1       1/2 +      -3.37822505e+04    Coulomb          3.98321466e+07  9.63493521e-10    6.83151383e-07
                                               Babushkin        9.77091676e+06  2.36347167e-10    2.78493684e-06
         2       7/2 +      -3.37696868e+04    Coulomb          8.97845000e+14  2.17178313e-02    3.03074428e-14
                                               Babushkin        8.97845000e+14  2.17178313e-02    3.03074428e-14
         3       5/2 + 


... and which shows all the intermediate output of the computations. First, all the initial-state levels are calculated *self-consistently* due to the default values for such SCF and configuration-interaction computations, and similarly for all final-state levels. These computations include the definition of the CSF basis, the generation of *start* orbitals as well as the computation of the angular coefficients, the SCF interation and up to the set-up and diagonalization of the corresponding Hamiltonian matrix. Obviously, this gives rise to a full representation of all atomic levels of interest.

These levels are then utilized to compute the (multipole) transitions amplitudes and rates. All final results are also tabulated in a neat format. However, since many of the internal tables of the JAC toolbox are *wider* than the standard widths of these notebooks, we shall first **close** our .txt summary file:


In [11]:
JAC.define("print summary: close", "")


This summary file **06-output-radiative.txt** can now be found in the current directory. As seen from these tables, the transition energies agree within 3..4 digits with a similar computation by other atomic structure codes, such as GRASP. Apart from the level splitting among the low-lying $3s 3p^6$, $3s^2 3p^4 3d$  $J=1/2$ levels, further comparision can be made also with the evaluated data from the NIST Spectroscopic Database, for instance, for the excitation energies from the $3s^2 3p^5$  $J = 3/2$ ground level.